<img style="float: right;"  src="images/LogoP.jpg" width="200">

# DC Module Reference

This a Jupyter Notebook Reference document for the SLab projects

Version 1.0 (4/5/2018) License information is at the end of the document

---

This **DC Module Reference Notebook** describes the **DC Module** commands.  
Those are the commands contained in the **dc.py** source file.


## Import and board connection

In order to use the DC commands, we first need to import the DC module. This is an interactive módule, so, in order to execute the code examples, you should import it executing the cell below.

We will also import the main **SLab** module as it includes some needed commands.

In [ ]:
# Import the main SLab module
import slab

# Import the DC module
import slab.dc as dc

If you want to try commands that interact with the **hardware board** using the examples in this document, you will need to connect to the board. You can use the following code cell for that. Note, however, that if you are using this document as reference while working on another SLab document, you can only have one connection, at the same time, with the board.

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

When you finish interacting with the board you can disconnect from it using the following code.

In [ ]:
# Disconnect from the board
slab.disconnect()

# Command Index

The DC SLab module contains several commands that ease the DC measurements. The following list contains all the commands in the DC module. You can use the hiperlinks to jump to one of the commands.

###  Two Terminal Device I-V Plots

[curveVI](#curveVI) : Positive I(V) curve  
[curveVIref](#curveVIref) : Positive and Negative I(V) curve  
[curveVIbridge](#curveVIbridge) : Bridge mode Positive and Negative I(V) curve  


###  Voltage Input-Output Plots

[curveVV](#curveVV) : Basic Vo(vi) curve  
[curveVVref](#curveVVref) Vo(vi) curve with reference voltage  
[curveVVbridge](#curveVVbridge) Bridge mode Vo(Vi) curve  
[hystVVcurve](#hystVVcurve) Hysteresis Vo(Vi) curve  

### Current Output Transfer Plots

[transferCurveVI](#transferCurveVI) : Transfer curve with input voltage and output current  
[transferCurveII](#transferCurveII) : Transfer curve with input and output currents  

Previous **SLab** versions had also **device curve** functions. Those are currently removed from the **DC** module. Their functionality is now in notebooks contained in the **Demo** folder.

In [ ]:
# Disconnect from the board
slab.disconnect()

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="8"> Two Terminal Device I-V Plots </font> 
<BR>
</div>

Commands on this section obtain the I-V characteristics of a two terminal component.

![IV Base](images/dc/iv_base.png)

A two terminal device can usually be characterized in DC by its I-V characteristic. That is the measurement of the device current for each possible voltage at its terminals.

The three commands curveVI, curveVIref and curveVIbridge are useful to obtain the I(V) DC curve of a two terminal device under test (DUT) in different scenarios.

<a id='curveVI'></a>

## curveVI

Perform a I(V) curve for a two terminal component

>**curveVI(v1,v2,vi,r,wt,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial voltage of DAC1 (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ r  : Resistor value in kohms (defaults to 1k)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)   

This command measures the current &I_D$ against voltage $V_D$ curve of a two terminal device under test (DUT) for positive voltages. In order to perform the measurement, the negative terminal of the device shall be connected to **GND** and the positive terminal to **ADC2**. A resistor shall also be connected between the **DAC1** output and **ADC1** input and the positive terminal of the DUT as shown in the following figure:

![curveVI](images/dc/curveVI.png)

The command will set **DAC1** voltage between **v1** and **v2** with **vi** step increments. If **vi** is omitted, it defaults to 0.1 V. A wait of **wt** seconds is included between each new **DAC1** value and the measurement on **ADC1** and **ADC2**. If **wt** is omitted it defaults to 0.1s.
When the measurement ends, **DAC1** voltage is set to zero to minimize power consumption.

From the measurements, current at the **DUT** is computed and the $I_D(V_D)$ curve is drawn

$\qquad V_D = V_{ADC1} \qquad I_D = \frac{V_{ADC1}-V_{ADC2}}{R}$

Note that we use **ADC1** connected to **DAC1** output. In theory we could obtain $I_D$ from $V_{DAC1}$ and $V_{ADC2}$ not needing the **ADC1** connection. In practice it is a good idea to obtain voltage values for the same kind of device (ADC in this case) if there is the possibility to substract them when they are nearly equal. In our case, using two ADCs give a better current measurement around zero than using only one ADC and the set DAC value. 
Moreover, you are not guaranteed to really have the set value at the DAC output so, specially at the end ranges of the DAC. Using an ADC guarantees that you know the voltage at the node.

Note also that we don't set the **DUT** input voltage range with the **v1** and **v2** parameters. Those parameters set the voltage range at the resistor input, the more current the device consumes, the less device voltage range we get. It is tempting to use a low value resistor to increase the voltage range, but there are two problems using this approach:

* Setting a low value resistor gives a small $V_{ADC1} - V_{ADC2}$ drop on the resistor. That limits the accuracy of the current measurement.

* Resistor **R** limits the current at the device. In order to guarantee that the device is not damaged and that the buffering circuit doesn't reach its compliance limits, it is recommended to limit the maximum current. We can calculate the limit, for the worst case as:

$\quad I_{DUT\;MAX} = \frac{V_{DAC1\;MAX}}{R} = \frac{v2}{R}$

You need to ensure that **DAC1** buffer circuit can provide the $I_D$ max current.

The command only works for positive $V_D$ voltages. In order to obtain I(V) curves that include negative voltages use the [curveVIref](#curveVIref) or [curveVIbridge](#curveVIbridge) instead.

If the optional parameter **returnData** is **True** or **setPlotReturnData** was called previously with a **True** argument, the command return a two element tuple with numpy arrays of:

* $V_D$ values
* $I_D$ values

---

**Example**  
Measure the forward curve of a 1N4148 diode.

---

We will connect the diode as **DUT** implementing the following circuit:

![curveVI Example](images/dc/curveVI_example.png)

The following **code cell** will generate the I(V) curve. We sweep **DAC1** voltage between $0V$ and $3.2V$ and use the default values for **vi**, **r** and **wt**.

By using a $1k\Omega$ resistor we have limited the current on the diode to a maximum of $3.2mA$. In practice, as voltage on the diode increases with current, the maximum current is $2.5mA$. Observe also, that the drop on **r** limits the measured VDUT voltage to less than $0.7V$.

In [ ]:
# curveVI example
# 1N4148 diode curve

dc.curveVI(0.0,3.2)

<a id='curveVIref'></a>

## curveVIref

Plots I(V) Device Curve with reference voltage

>**curveVIref(v1,v2,vi,r,vr,wt,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial value of DAC1 (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ r  : Resistor value in kohms (defaults to 1k)  
>$\quad$ vr : Reference voltage (defaults to Vdd/2)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)  

This command measures the current $I_D$ against voltage $V_D$ curve of a two terminal device under test (DUT).  
In order to perform the measurement, the negative terminal of the device shall be connected to the central point of two equal **r** value resistors connected to Vdd and GND and to **ADC2**.  
**DAC1** and **ADC1** shall be connected to the positive terminal of the **DUT**.   
The following figure shows the circuit connections:  

![curveVIref](images/dc/curveVIref.png)

By connecting the negative terminal of the **DUT** as shown, negative voltages and currents can be measured. As the total voltage range has not changed compared to the [curveVI](#curveVI) command, the measurement of negative voltages reduces the positive voltage range.

The command will set **DAC1** voltage between **v1** and **v2** with **vi** step increments. If **vi** is omitted, it defaults to $0.1 V$. A wait of **wt** seconds is included between each new **DAC1** value and the measurement on **ADC1** and **ADC2**. If **wt** is omitted it defaults to $0.1s$.
When the measurement ends, **DAC1** voltage is set to **Vdd/2** to minimize power consumption.

From the measurements, voltage and current at the **DUT** is computed and the $I_D(V_D)$ curve is drawn.

$\qquad V_D = V_{ADC1} - V_{ADC2} \qquad I_D = 2 \frac{V_{ADC2}-V_{DD}/2}{R}$

Results obtained by using this command depend on both resistors having the same **r** value. If this condition is not met, an error in the current value will be shown on the curve.

As in the [curveVI](#curveVI) command, **r** needs to be chosen value both to obtain good current accuracy and to limit the current in the **DUT**. In this case, also, the circuit always consumes current as the two resistors are always connected between $V_{DD}$ and $GND$. Default current consumption of the circuit, when $I_D$ is zero is:

$\qquad I_{Base} = \frac{V_{DD}}{2R}$

Current trough the **DUT** is bounded by:

$\qquad |I_D|_{MAX} = \frac{V_{DD}}{R}$

You need to ensure that **DAC1** buffer circuit can provide this current.

The command requires that both resistors are equal and that the **V_{DD}** value is known. If not, current won’t show as zero for zero voltage. In order to increment the precision of the measurement you can provide the reference voltage using the optional **vr**parameter. In that case, the current is calculated as:

$\qquad I_D = 2\frac{V_{ADC2}-V_r}{R}$

In order to obtain this Vr voltage you just need to measure the voltage at **ADC2** after disconnecting **DAC1** from the circuit.

If parameter **returnData** is **True** or **setPlotReturnData** was called with a **True** parameter, the command return a two element tuple with numpy arrays of:

* $V_D$ values
* $I_D$ values


---

**Example**  
Measure the I(V) curve of two antiparallel 1N4148 diodes.

---

We will set the pair of diodes as the **DUT** and implement the following circuit:

![curveVIref Example](images/dc/curveVIref_example.png)

The following **code cell** will generate the I(V) curve. We sweep **DAC1** voltage between $0V$ and $3.2V$ and use the default values for **vi**, **r** and **wt**.

If current is no exactly 0 at 0 voltage. That could probably be due to using two not exactly equal $1k\Omega$ resistors. In order to increase the current measurement precision, the real reference voltage can be measured and be indicated as the optional **vr** parameter.

Current is limited to $3.2mA$ due to the use of $1k\Omega$ resistors. The voltage drop on the diodes limits the maximum current below $2mA$.

In [ ]:
# curveVIref example
# Two 1N4148 in antiparallel curve

dc.curveVIref(0.0,3.2)

<a id='curveVIbridge'></a>

## curveVIbridge

Plots I(V) Device Curve in bridge configuration

>**curveVIbridge(v1max,v2max,vi=0.1,vmin,r,wt,returnData)**  
>Required parameters:   
>$\quad$ v1max : Max voltage at DAC 1 (in Volt)  
>$\quad$ v2max : Max voltage at DAC 2 (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ vmin : Min DAC voltage (Defaults to 0 V)  
>$\quad$ r  : Resistor value in kohms (defaults to 1k)  
>$\quad$ vr : Reference voltage (defaults to Vdd/2)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)  

The previous [curveVIref](#curveVIref) command enables us to obtain the $I(V)$ curve of a two terminal device including negative voltages at expense of the positive voltage range. By using a **bridge configuration**, this command enables us to measure the negative voltage range of a device without compromising the positive range. In order to perform the measurement, the negative terminal of the device shall be connected to **DAC2** and **ADC3** and the positive terminal to **ADC2**. A resistor shall also be connected between the **DAC1** output and **ADC1** input and the positive terminal of the **DUT** as shown in the following figure:

![curveVIbridge](images/dc/curveVIbridge.png)

The command will obtain the $I(V)$ curve in two stages. In the first stage, **DAC2** voltage is set to **vmin** and **DAC1** voltage changes between **vmin** and **v1max** with **vi** step increments. In the second stage, **DAC1** voltage is set to **vmin** and **DAC2** voltage changes between **vmin** and **v2max** with **vi** step increments. If **vi** is omitted, it defaults to $0.1 V$.

A wait of **wt** seconds is included between each new DAC value and the measurement on the ADCs. If **wt** is omitted it defaults to $0.1s$.

When the measurement ends, both DACs are set to zero to minimize power consumption.

From the measurements, voltage and current at the DUT is computed and the $I_D(V_D)$ curve is drawn.

$\qquad V_D = V_{ADC2} - V_{ADC3}  \qquad  I_D = \frac{V_{ADC1}-V_{ADC2}}{R}$

As in the two previous command, the **r** value needs to be chosen both to obtain good current accuracy and to limit the current in the **DUT**. Current trough the DUT is limited to:

$\qquad |I_D|_{MAX} = \frac{V_{DD}}{R}$

You need to ensure that **DAC1** buffer circuit can provide this current.

If parameter **returnData** is **True** or **setPlotReturnData** was called previously with a **True** parameter, the command return a two element tuple with numpy arrays of:

* $V_D$ values
* $I_D$ values


---

**Example**  
Obtain the $I(V)$ response of a white led both for positive and negative voltages

---

The threshold of a white LED is too high to use the [curveVIref](#curveVIref) command, so we will use the bridge configuration. The following circuit shows the circuit to implement:

![curveVIbridge Example](images/dc/curveVIbridge_example.png)

The following **code cell** will generate the $I(V)$ curve.

Maximum current is limited to $3.2mA$ due to the use of a $1k\Omega$ resistor. As voltage drop in a blue LED is high (about 2.5V) current is limited to about $0.5mA$. That limit can be calculated:

$\qquad I_{MAX} = \frac{V_{DAC1\;MAX} - V_{DAC2\;MIN} - V_D}{R}$

In [ ]:
# curveVIbridge example
# White LED I(V) curve

dc.curveVIbridge(3.2,3.2,vmin=0.2)

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="8"> Voltage Input-Output Plots </font> 
<BR>
</div>

Commands in this section obtain the voltage DC response of a circuit under test (CUT). That is, they obtain the output voltage for a sequence of input voltage values.

![Vo(Vi)](images/dc/vo_vi.png)

The following three commands [CurveVV](#CurveVV), [CurveVVref](#CurveVVref) and [HystVVcurve](#HystVVcurve) are useful in order to obtain the DC output voltage to input voltage curve of a circuit under test (CUT).

<a id='curveVV'></a>

## curveVV

Plots a V(V) transfer curve

>**curveVV(v1,v2,vi,wt,adc2,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial value (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ adc2 : Use ADC2 to improve accuracy (Defaults False)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)  

Draw voltage set with **DAC1** against voltage measured with **ADC1**.

Measurements start with **DAC1** at **v1** voltage and end at **v2** voltage. Voltage is increased in a **vi** step between one measurement and the next one. If **vi** is omitted, it defaults to $0.1 V$.

Between the change of the DAC voltage and the measurement with the ADCs the module waits **wt** seconds. If **wt** is omitted it defaults to $0.1s$.

This command is useful for obtaining the DC voltage response of a circuit. The circuit under test (CUT), as shown in the following figure, is driven by an input voltage generated by **DAC1** and its output is read at **ADC1**.

![curveVV](images/dc/curveVV.png)

If optional parameter **adc2** is **True**, **ADC2** will be used to measure the Vi voltage. It is useful if you want to be sure of the Vi voltage really set by **DAC1**. The following figure shows the proper connections in this case.

![curveVV 2](images/dc/curveVV2.png)

If optional parameter **returnData** is **True** or **setPlotReturnData** was called previously with a **True** parameter, the command return a two element tuple with numpy arrays of:

* $V_i$ values
* $V_o$ values

The same functionality of this command can be obtained with the **dcSweepPlot** command contained in the main **slab module**. This command, however, is more simple and having it on the DC module gives this module a more complete set of commands.

---

**Example**  
Measure a voltage drop circuit

---

The voltage drop circuit is shown in the following figure. For **Vi** input voltages below about $0.5V$, **D1** does not operate and output voltage is zero. For voltages over about $0.5V$, the output voltage is decreased respect to the input voltage due to the **D1** voltage drop. 

![curveVV Example](images/dc/curveVV_example.png)

In order to test the $Vo(Vi)$ curve of the circuit we can execute the following **code cell**.

Observe that the diode drop is $0.5V$ only at the start, and that it increase to arround $0.65V$ at the end.


In [ ]:
# curveVV example
# Voltage drop circuit Vo(Vi) plot

dc.curveVV(0.0,3.0,adc2=True)

<a id='curveVVref'></a>

## curveVVref

Plots a V(V) transfer curve with reference

>**curveVVref(v1,v2,vi,wt,adc3,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial value (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ adc3 : Use ADC3 to sense Vi (Defaults to False)    
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)  

Obtain a circuit under test (CUT) $Vo(Vi)$ voltage taking into account a not grounded reference for the circuit.

Set circuit input voltage with **DAC1** and measures output voltage with **ADC1**.
Reference voltage of the circuit is measured with **ADC2**.
Input $V_i$ and output $V_o$ voltages are calculated as:

$\qquad V_{Ref} = V_{ADC2} \qquad V_i = V_{DAC1} - V_{Ref} 
 \qquad V_o = V_{ADC1} - V_{Ref}$
 
Measurements start with **DAC1** at **v1** voltage and end at **v2** voltage. Voltage is increased in a **vi** value between one measurement and the next one. If **vi** is omitted, it defaults to $0.1 V$.

Between the change of the DAC voltage and the measurement with the ADCs the module waits **wt** seconds. If **wt** is omitted it defaults to $0.1 s$.
From the measurements, $V_i$ and $V_o$ are calculated and plotted.

This command is useful for obtaining the DC voltage response of a circuit that uses a not grounded reference. The circuit under test (CUT), as shown in the following figure, is driven by an input voltage generated by **DAC1**, its output is read at **ADC1** and the reference is read at **ADC2**.

![curveVVref](images/dc/curveVVref.png)

If optional parameter **adc3** is **True**, **ADC3** will be used to measure the $V_i$ voltage. It is useful if you want to be sure of the $V_i$ voltage really set by **DAC1**. The following figure shows the proper connections in this case.

![curveVVref 2](images/dc/curveVVref2.png)

If parameter **returnData** is **True** or **setPlotReturnData** was called previously with a **True** parameter, the command return a two element tuple with numpy arrays of:

* $V_i$ values
* $V_o$ values


---

**Example**  
Obtain the $V_o(V_i)$ curve of a 10 gain operational amplifier inverter circuit with reference. 

---

The following figure shows the circuit we want to test:

![curveVVref Example](images/dc/curveVVref_example.png)

The circuit uses one of the two full-rail operational amplifiers included in the **MCP6002** integrated circuit. Resistors **R3** and **R4** set the gain between $V_i$ and $V_o$ to -10. Resistors **R1** and **R2** set the reference voltage for the circuit to halfway between $V_{DD}$ and $GND$.

The following **code cell** performs the measurement.

We see that output is zero when input is zero (respect to the $V_{DD}/2$ reference). We can also can check that the gain is really -10.

In [ ]:
# curveVVref example
# Inverting Operational Amplifier Circuit

dc.curveVVref(0.0,3.0,adc3=True)

<a id='curveVVbridge'></a>

## curveVVbridge

Plots a V(V) transfer curve in bridge configuration

>**curveVVbridge(vp,vn,vi,vmin,wt,returnData)**  
>Required parameters:   
>$\quad$ vp : Maximum positive voltage (in Volt)  
>$\quad$ vn : Maximum negative voltage (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ vmin : Minimum DAC voltage (Defaults to 0.0)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)

This command obtains a circuit under test (CUT) $V_o(V_i)$ using a bridge configuration.

![curveVVbridge](images/dc/curveVVbridge.png)

The command generates two sweeps:

In the first sweep, **DAC2** is set to **vmin** and **DAC1** sweeps from **vmin** to **vp** in **vi** increments.
In the second sweep, **DAC1** is set to **vmin** and **DAC2** sweeps from **vmin** to **vn** in **vi** increments.

Input and output voltages are defined as:

$\qquad V_i = V_{ADC3}-V_{ADC4} \qquad V_o = V_{ADC1} - V_{ADC2}$


---

**Example**  
Obtain the Vo(Vi) curve of a diode bridge

---

The following figure shows the circuit we want to test:

![curveVVbridge Example](images/dc/curveVVbridge_example.png)

We have added a $100 k\Omega$ **R2** resistor from the output negative terminal to ground because, when no diode conducts, the output floats and that will provide bad measurements.

We increase the number of readings to reduce measurement noise and perform a $V_o(V_i)$ bridge curve. We also set the minimum DAC voltage to $0.2V$ to prevent working on the lower end of the DAC range.


In [ ]:
# curveVVbridge example
# Diode bridge circuit

# Increase number of readings
old = slab.setDCreadings(400)

# Plot curve
dc.curveVVbridge(3.2,3.2,vmin=0.2)

# Restore number of readings
slab.setDCreadings(old) 

<a id='hystVVcurve'></a>

## hystVVcurve

V(V) Transfer Hysteresis Curve

>**hystVVcurve(v1,v2,vi,wt,adc2,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial value (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ adc2 : Use adc2 to measure vi (Defaults to False)
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)

Some circuits have a DC response that changes if the input voltage is rising or falling. The hysteresis curve command is similar to the [curveVV](#curveVV) command, but measures the response of the circuit both in the forward direction with **DAC1** voltage going from **v1** to **v2** and in the back direction with **DAC1** voltage going from **v2** to **v1**. In both cases voltage is measured at **ADC1**.

If **adc2** is **True**, **ADC2** will be used to measure $V_i$

If **returnData** is **True** or **setPlotReturnData** was called previously with a **True** parameter, the command return a four element tuple with numpy arrays of:

* $V_i$ forward values
* $V_o$ forward values
* $V_i$ reverse values
* $V_o$ reverse values


---

**Example**  
Measure a hysteresis inverting comparator curve.

---

The circuit to test is shown in the following figure. **R1** and **R2** set the center of the comparator change at $V_{DD}/2$ that is about $1.65V$ for a 3.3V powered board. **R3** sets the hysteresis of the comparator.

![hystVVcurve Example](images/dc/hystVVcurve_example.png)

In order to see the I/O curve we can issue the command in the **code cell** below that sweeps **DAC1** between $0V$ and $3V$ both rising (Forward) and falling (Back).




In [ ]:
# hystVVcurve example
# Hysteresis comparator

dc.hystVVcurve(0.0,3.0,adc2=True)

It seems that forward (blue) and back (blue) curves are not vertical. This is due to the default $0.1V$ step of the previous command. That means that we have, for instance on the input in the back direction, one point at $1.6V$ and another at $1.5V$.
If we want more precise curve around the change zone of the comparator we can perform a finer $10mV$ step between $1.2V$ and $2V$.

In [ ]:
# hystVVcurve example
# Hysteresis comparator detail

dc.hystVVcurve(1.2,2.0,0.01,adc2=True)

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="8"> Current Output Transfer Plots </font> 
<BR>
</div>

The following two commands [transferCurveVI](#transferCurveVI) and [transferCurveII](#transferCurveII) are designed to measure circuits whose output is defined on current instead of voltage.

With those two command, added to the previous ones, we obtain the following table of circuit transfer curves:

<img  src="images/dc/transferTable.png" width="400">

Those commands don’t include all possibilities. For instance, there is no command for circuits with current input and voltage output. Moreover, hysteresis and negative ranges are only measured in voltage to voltage circuits. As all this commands rely on the Basic DC commands, it is possible to define any new commands if the ones defined in the DC module are not suited for a particular measurement.

<a id='transferCurveVI'></a>

## transferCurveVI

Plots a I(V) Transfer Curve

>**transferCurveVI(v1,v2,vi,ro,adc3,wt,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial value of DAC1 (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ ro : Resistor Ro value in kohms (defaults to 1k)  
>$\quad$ adc3 : Use ADC3 (defaults to False)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)

This command measures the output current of a circuit under test (CUT) against its input voltage. The following figure shows the setup of connections to perform the measurement. 

![transferCurveVI](images/dc/transferCurveVI.png)

Depending on the circuit $R_i$ can be omitted. Sometimes, however, it will interest to limit the input current on the circuit using this resistor. Either way, it is not used in the calculations.

The command will set **DAC1** voltage between **v1** and **v2** with **vi** step increments. If **vi** is omitted, it defaults to $0.1 V$. A wait of **wt** seconds is included between each new **DAC1** value and the measurement on **ADC1** and **ADC2**. If **wt** is omitted it defaults to $0.1s$.  
When the measurement ends, **DAC1** voltage is set to zero to minimize power consumption.

If the optional parameter **adc3** is **True**, the circuit used will measure both voltages on $R_o$ using ADC readings. To obtain both voltages the following circuit will be used.

![transferCurveVI2](images/dc/transferCurveVI2.png)

From the measurements, a $I_o(V_i)$ curve will be shown.

If parameter **returnData** is **True** or **setPlotReturnData** was called previously with a **True** parameter, the command return a two element tuple with numpy arrays of:

* $V_i$ values
* $I_o$ values



---

**Example**  
Measure the collector current against the base voltage on a BC547 bipolar junction transistor (BJT)

---

We will setup the following circuit:

![transferCurveVI Example](images/dc/transferCurveVI_example.png)

In order to obtain the Ic(Vbe) curve we execute the following **code cell**. First we increase the number of measurements to average to increase the accuracy and then we draw the curve. We have used a fine $20mV$ step to provide a better representation at the lower range of the curve.



In [ ]:
# transferCurveVI example
# Ic(Vbe) BJT curve

# Increasae number of readings
old = slab.setDCreadings(100) 

# Plot the curve
dc.transferCurveVI(0.5,3.0,0.02)

# Restore number of readings
slab.setDCreadings(old)

As **ro** is $1k\Omega$ and the transistor saturates at $V_{CE}$ of about $0.2V$, maximum current before saturation is about $3.1mA$. So, points above that current are not guaranteed to be in the linear active region of the BJT.

If we want to operate at higher currents we can reduce **ri** and **ro**. We want to limit the collector current to $50mA$ because this transistor cannot operate above $100mA$ and because the nucleo board cannot provide more than $300mA$ to all connected elements. Using a $68\Omega$ ro value we will enter saturation at about $45mA$. We will also decrease **ri** to $8.2k\Omega$ so that there is enough base current to have a high collector current.

The following **code cell** performs the measurements. As the **vi**, **wt** and **ro** are not in the proper order, the name of the parameter shall be included. That way you don't need to remember the order of the optional parameters. Note also that **ro** is given in $k\Omega$. This time we don't need to use a $20mV$ step because we start on **DAC1** at $1V$ instead of $0.5V$.

In [ ]:
# transferCurveVI example
# Ic(Vbe) BJT curve

# Increasae number of readings
old = slab.setDCreadings(100) 

# Plot the curve
dc.transferCurveVI(1.0,3.0,vi=0.1,wt=0.1,ro=0.068)

# Restore number of readings
slab.setDCreadings(old)

<a id='transferCurveII'></a>

## transferCurveII

Plots a I(I) Transfer Curve

>**transferCurveII(v1,v2,vi,r1,r2,adc3,adc4,wt,returnData)**  
>Required parameters:   
>$\quad$ v1 : Initial value of DAC1 (in Volt)  
>$\quad$ v2 : End of range (in Volt)  
>Optional parameters:   
>$\quad$ vi : Step (defaults to 0.1V)  
>$\quad$ r1 : Resistor R1 value in kohms (defaults to 1k)  
>$\quad$ r2 : Resistor R2 value in kohms (defaults to 1k)  
>$\quad$ adc3 : Use ADC3 to measure r2 high side (defaults to False)  
>$\quad$ adc4 : Use ADC4 to measure DAC1 (defaults to False)  
>$\quad$ wt : Waiting time between steps (defaults to 0.1s)  
>$\quad$ returnData : Enable return of plot data (Defaults to False)  
>Returns plot data if enabled (see also setPlotReturnData)  

This command is similar to the previous one and measures the output current of a circuit under test (CUT) against its input current. The connections are the same than in the previous case.

![transferCurveII](images/dc/transferCurveII.png)

As we need **R1** to compute the input current, this time, this resistor cannot be omitted. 

The command will set **DAC1** voltage between **v1** and **v2** with **vi** step increments. If **vi** is omitted, it defaults to $0.1 V$. A wait of **wt** seconds is included between each new **DAC1** value and the measurement on the ADCs. If **wt** is omitted it defaults to $0.1s$.
When the measurement ends, **DAC1** voltage is set to zero to minimize power consumption.

If the **adc3** parameter is **True**, the **ADC3** will be used to measure the high side of **R2** instead of assuming it to be $V_{DD}$. If the **adc4** parameter is **True**, the **ADC4** will be used to measure the **DAC1** output instead of relying on the DAC programmed value. The following figure shows those optional connections.

![transferCurveII2](images/dc/transferCurveII2.png)

From the measurements, a $I_o(I_i)$ curve will be shown.

If **returnData** is **True** or **setPlotReturnData** was called previously with a **True** parameter, the command return a two element tuple with numpy arrays of:

* $I_i$ values
* $I_o$ values


---

**Example**  
Measure the collector current against the base current on a BC547 transistor

---

We will setup a similar circuit as in the [transferCurveVI](#transferCurveVI) example.

![transferCurveII Example](images/dc/transferCurveII_example.png)

In order to obtain the $I_C(V_{BE})$ curve we use the following **code cell**. Parameteres **r2** and **wt** use their default values of $1k\Omega$ and $0.1s$.

We obtain the following graph where we observe a constant $\beta$ ratio between $I_C$ and $I_B$ up to the point where the transistor enters in the saturation region.

In [ ]:
# transferCurveII example
# Ic(Ib) BJT curve

# Obtain the transfer curve
dc.transferCurveII(0.5,2.0,0.1,100.0)

At low currents, the $\beta$ of the transistor is reasonably constant. This parameter, however, degrades at high currents. If we want to operate at higher currents we can reduce **ri** and **ro**. Using a $68\Omega$ ro value we will enter saturation at about $45mA$. We will also reduce **ri** to $8.2k\Omega$ to provide enough base current.

Before issuing the **transferCurveII** command we will also increase the number of readings to reduce the measurement errors.

We can see that $\beta$ degrades at currents over $30mA$.

In [ ]:
# transferCurveII example
# Ic(Ib) BJT curve

# Increase number of readings
old = slab.setDCreadings(50) 

# Draw the curve
dc.transferCurveII(1.0,3.0,0.1,8.2,0.068)

# Restore number of readings
slab.setDCreadings(old) 

## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">